# Coursera capstone project

#### Find the best location to open a Boba milk tea shop

In [2]:
#importing libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('folium imported')

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
# one more tool
!pip install bs4
from bs4 import BeautifulSoup


  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


#### Find the neightborhoods in San Fernando Valley by scaping data from wikipedia page

In [181]:
url = 'https://en.wikipedia.org/wiki/Category:Communities_in_the_San_Fernando_Valley'

In [182]:
#usingbeautiful soup to extract data
data = requests.get(url).text
soup = BeautifulSoup(data,'html.parser')

In [183]:
print(soup.prettify()) # Used to find the class

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Communities in the San Fernando Valley - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6391fc30-0eaf-4dfc-95f1-24bf44bf87c8","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Communities_in_the_San_Fernando_Valley","wgTitle":"Communities in the San Fernando Valley","wgCurRevisionId":595236711,"wgRevisionId":595236711,"wgArticleId":16201279,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["San Fernando Valley","Populated places in Los An

In [184]:
neighborhood = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("a"):
    neighborhood.append(row.text)

In [185]:
# put the data into a pandas dataframe
df = pd.DataFrame({'neighborhoods':neighborhood})
print(df.head())
df.shape

              neighborhoods
0       Arleta, Los Angeles
1       Burbank, California
2     Calabasas, California
3  Canoga Park, Los Angeles
4   Chatsworth, Los Angeles


(31, 1)

#### Find the coordinates of each neighborhood

In [76]:
def findCoord(name):
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(name)
    lat = location.latitude
    long = location.longitude
    return long, lat

In [186]:
# you dont wanna rerun this
lat = []
long = []
for address in df['neighborhoods'].tolist():
    lat.append(findCoord(address)[1])
    long.append(findCoord(address)[0])

In [187]:
# put the found location information to pandas dataframe
df['Latitude'] = lat
df['Longitude'] = long

In [188]:
df.rename(columns={'neighborhoods':'Neighborhoods'},inplace = True)
df.head()

,Neighborhoods,Latitude,Longitude
0,"Arleta, Los Angeles",34.241327,-118.432205
1,"Burbank, California",34.181648,-118.325855
2,"Calabasas, California",34.144664,-118.644097
3,"Canoga Park, Los Angeles",34.201078,-118.597826
4,"Chatsworth, Los Angeles",34.259571,-118.602325


#### Creating a map of San Fernando Valley

In [89]:
address = 'San Fernando, California'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Fernando Valley {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Fernando Valley 34.28497, -118.437652.


In [96]:
sf_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhoods']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(sf_map) 



In [97]:
sf_map

#### Find the venues with Foursquare API

In [98]:
#Define Foursquare Credentials
CLIENT_ID = 'NGV3SZ5M5UJSNNGGIIZEVLIPIUZEVLLL5ARE1HDPW3PRRKD1' # your Foursquare ID
CLIENT_SECRET = '2QCX1X5HSGDOG33DJQ3R3QDK5VYF2AMLII2E2PARTINWHQZR' # your Foursquare Secret
VERSION = '20200513' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NGV3SZ5M5UJSNNGGIIZEVLIPIUZEVLLL5ARE1HDPW3PRRKD1
CLIENT_SECRET:2QCX1X5HSGDOG33DJQ3R3QDK5VYF2AMLII2E2PARTINWHQZR


In [103]:
# Find the top 50 venues in one neighborhood
LIMIT = 50 
radius = 500 

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'],df['Neighborhoods']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=NGV3SZ5M5UJSNNGGIIZEVLIPIUZEVLLL5ARE1HDPW3PRRKD1&client_secret=2QCX1X5HSGDOG33DJQ3R3QDK5VYF2AMLII2E2PARTINWHQZR&v=20200513&ll=34.1684364,-118.6058382&radius=500&limit=50'

In [104]:
# check the result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed2320c216785001b3e8afe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Woodland Hills-Warner Center',
  'headerFullLocation': 'Woodland Hills-Warner Center, Los Angeles',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 57,
  'suggestedBounds': {'ne': {'lat': 34.1729364045, 'lng': -118.60040956058509},
   'sw': {'lat': 34.163936395499995, 'lng': -118.6112668394149}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55007908498e4a53c3de81f2',
       'name': 'Ralphs Fuel Center',
       'location': {'address': '21909 Ventura Blvd',
        'crossStreet': 'at Topanga Canyon Blvd',
        

In [107]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ralphs Fuel Center,Gas Station,34.168365,-118.605601
1,The High Tea Cottage,Tea Room,34.167430,-118.605443
2,Starbucks,Coffee Shop,34.169307,-118.605482
3,Monty's Prime Steaks & Seafood,Steakhouse,34.167958,-118.606351
4,Sprouts Farmers Market,Grocery Store,34.169436,-118.603069


In [112]:
## repeat it for the rest of the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius,limit):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
sf_venues_df = getNearbyVenues(names= df['Neighborhoods'],
                               latitudes = df['Latitude'],
                               longitudes = df['Longitude'],
                               radius = 500,limit = 50)

Arleta, Los Angeles
Burbank, California
Calabasas, California
Canoga Park, Los Angeles
Chatsworth, Los Angeles
Encino, Los Angeles
Glendale, California
Granada Hills, Los Angeles
Hidden Hills, California
Lake Balboa, Los Angeles
Lake View Terrace, Los Angeles
Mission Hills, Los Angeles
North Hollywood, Los Angeles
Northridge, Los Angeles
Pacoima, Los Angeles
Panorama City, Los Angeles
Porter Ranch, Los Angeles
Reseda, Los Angeles
San Fernando, California
Sherman Oaks, Los Angeles
Studio City, Los Angeles
Sun Valley, Los Angeles
Sunland-Tujunga, Los Angeles
Sylmar, Los Angeles
Tarzana, Los Angeles
Toluca Lake, Los Angeles
Universal City, California
Van Nuys, Los Angeles
West Hills, Los Angeles
Winnetka, Los Angeles
Woodland Hills, Los Angeles


In [114]:
sf_venues_df.head() # take a look of our results

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Arleta, Los Angeles",34.241327,-118.432205,Back To The Future Filming Location - McFly's ...,34.243429,-118.433655,Historic Site
1,"Arleta, Los Angeles",34.241327,-118.432205,Edwards Cinema,34.241197,-118.430284,Movie Theater
2,"Arleta, Los Angeles",34.241327,-118.432205,Canterbury & Kelowna,34.239525,-118.435370,Movie Theater
3,"Burbank, California",34.181648,-118.325855,Santoro's Submarine Sandwiches,34.182351,-118.327926,Deli / Bodega
4,"Burbank, California",34.181648,-118.325855,Half Baked Co.,34.183009,-118.324820,Bakery


In [116]:
# check the size
sf_venues_df.shape

(618, 7)

In [117]:
sf_venues_df.groupby('Neighborhood').count() # number of venues for each neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Arleta, Los Angeles",3,3,3,3,3,3
"Burbank, California",12,12,12,12,12,12
"Canoga Park, Los Angeles",19,19,19,19,19,19
"Chatsworth, Los Angeles",17,17,17,17,17,17
"Encino, Los Angeles",23,23,23,23,23,23
"Glendale, California",35,35,35,35,35,35
"Granada Hills, Los Angeles",23,23,23,23,23,23
"Hidden Hills, California",2,2,2,2,2,2
"Lake Balboa, Los Angeles",7,7,7,7,7,7


In [160]:
temp = sf_venues_df.groupby('Neighborhood')
temp.shape

AttributeError: Cannot access attribute 'shape' of 'DataFrameGroupBy' objects, try using the 'apply' method

#### Further analysize the neighborhoods with desired features

In [157]:
# using one hot encoding
onehot_sf = pd.get_dummies(sf_venues_df[['Venue Category']],prefix="",prefix_sep = "")
onehot_sf["Neighborhoods"] = sf_venues_df['Neighborhood']

onehot_sf.head()

(618, 161)

In [133]:
# sort the table by asian food/asian retaurant
asia_food = onehot_sf[['Neighborhoods','Asian Restaurant']]

sort_sf = asia_food.sort_values(by='Asian Restaurant',axis =0,ascending =False)
sort_sf.head(15)

,Neighborhoods,Asian Restaurant
182,"North Hollywood, Los Angeles",1
234,"Northridge, Los Angeles",1
577,"Woodland Hills, Los Angeles",1
20,"Canoga Park, Los Angeles",1
74,"Glendale, California",1
220,"Northridge, Los Angeles",1
245,"Panorama City, Los Angeles",1
478,"Toluca Lake, Los Angeles",1
15,"Canoga Park, Los Angeles",1
413,"Sunland-Tujunga, Los Angeles",0


In [156]:
# find more about each neighborhood
sf_grouped =onehot_sf.groupby('Neighborhoods').mean().reset_index()
sf_grouped.head()


,Neighborhoods,ATM,American Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Big Box Store,Board Shop,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,English Restaurant,Event Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,Historic Site,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Latin American Restaurant,Liquor Store,Locksmith,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Music Venue,Nail Salon,New American Restaurant,Noodle House,North Indian Restaurant,Office,Optical Shop,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Racetrack,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,South American Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Stadium,State / Provincial Park,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tattoo Parlor,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Arleta, Los Angeles",0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,"Burbank, California",0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.083333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.083333,0.000000,0.0,0.000000,0.083333,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.000000,0.083333,0.0,0.0,0.000000,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.

In [136]:
num_top_venues = 5

for hood in sf_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = sf_grouped[sf_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arleta, Los Angeles----
           venue  freq
0  Movie Theater  0.67
1  Historic Site  0.33
2            ATM  0.00
3      Pet Store  0.00
4       Pharmacy  0.00


----Burbank, California----
            venue  freq
0  Sandwich Place  0.08
1          Bakery  0.08
2    Concert Hall  0.08
3             Spa  0.08
4       Racetrack  0.08


----Canoga Park, Los Angeles----
                       venue  freq
0           Asian Restaurant  0.11
1        Rental Car Location  0.11
2     Thrift / Vintage Store  0.11
3             Sandwich Place  0.05
4  South American Restaurant  0.05


----Chatsworth, Los Angeles----
                       venue  freq
0         Mexican Restaurant  0.18
1       Gym / Fitness Center  0.12
2  Cajun / Creole Restaurant  0.06
3                      Hotel  0.06
4       Fast Food Restaurant  0.06


----Encino, Los Angeles----
                 venue  freq
0  Japanese Restaurant  0.09
1          Pizza Place  0.09
2                  ATM  0.04
3         Burger Joint  0

In [139]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [154]:
# put all the information into one dataframe
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = sf_grouped['Neighborhoods']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Arleta, Los Angeles",Movie Theater,Historic Site,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop
1,"Burbank, California",Rental Service,Sandwich Place,Spa,Fast Food Restaurant,Concert Hall
2,"Canoga Park, Los Angeles",Rental Car Location,Asian Restaurant,Thrift / Vintage Store,Pharmacy,Café
3,"Chatsworth, Los Angeles",Mexican Restaurant,Gym / Fitness Center,Creperie,Cajun / Creole Restaurant,Spa
4,"Encino, Los Angeles",Japanese Restaurant,Pizza Place,ATM,Gym,Chinese Restaurant
5,"Glendale, California",Coffee Shop,Grocery Store,Discount Store,Automotive Shop,Shopping Plaza
6,"Granada Hills, Los Angeles",Pizza Place,Cosmetics Shop,Bakery,Insurance Office,Middle Eastern Restaurant
7,"Hidden Hills, California",Pool,Home Service,Yoga Studio,Frozen Yogurt Shop,Fried Chicken Joint
8,"Lake Balboa, Los Angeles",Park,Baseball Field,American Restaurant,Golf Course,Athletics & Sports
9,"Lake View Terrace, Los Angeles",Garden,Trail,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop


#### Finally Clustering the neighborhoods based on their similarity

In [176]:
# try 5 cluster first
# set number of clusters
kclusters = 7

sf_grouped_clustering = sf_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sf_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 1, 1, 1, 1, 1, 1, 6, 1, 0], dtype=int32)

In [177]:
# adding the clustering number to dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [178]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,5,"Arleta, Los Angeles",Movie Theater,Historic Site,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop
1,1,"Burbank, California",Rental Service,Sandwich Place,Spa,Fast Food Restaurant,Concert Hall
2,1,"Canoga Park, Los Angeles",Rental Car Location,Asian Restaurant,Thrift / Vintage Store,Pharmacy,Café
3,1,"Chatsworth, Los Angeles",Mexican Restaurant,Gym / Fitness Center,Creperie,Cajun / Creole Restaurant,Spa
4,1,"Encino, Los Angeles",Japanese Restaurant,Pizza Place,ATM,Gym,Chinese Restaurant


In [189]:
df = df.join(neighborhoods_venues_sorted.set_index('Neighborhoods'),on ='Neighborhoods')
print(neighborhoods_venues_sorted.head())
df.head()

   Cluster Labels             Neighborhoods 1st Most Common Venue  \
0               5       Arleta, Los Angeles         Movie Theater   
1               1       Burbank, California        Rental Service   
2               1  Canoga Park, Los Angeles   Rental Car Location   
3               1   Chatsworth, Los Angeles    Mexican Restaurant   
4               1       Encino, Los Angeles   Japanese Restaurant   

  2nd Most Common Venue   3rd Most Common Venue      4th Most Common Venue  \
0         Historic Site     Filipino Restaurant     Furniture / Home Store   
1        Sandwich Place                     Spa       Fast Food Restaurant   
2      Asian Restaurant  Thrift / Vintage Store                   Pharmacy   
3  Gym / Fitness Center                Creperie  Cajun / Creole Restaurant   
4           Pizza Place                     ATM                        Gym   

  5th Most Common Venue  
0    Frozen Yogurt Shop  
1          Concert Hall  
2                  Café  
3           

,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Arleta, Los Angeles",34.241327,-118.432205,5.0,Movie Theater,Historic Site,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop
1,"Burbank, California",34.181648,-118.325855,1.0,Rental Service,Sandwich Place,Spa,Fast Food Restaurant,Concert Hall
2,"Calabasas, California",34.144664,-118.644097,NaN,NaN,NaN,NaN,NaN,NaN
3,"Canoga Park, Los Angeles",34.201078,-118.597826,1.0,Rental Car Location,Asian Restaurant,Thrift / Vintage Store,Pharmacy,Café
4,"Chatsworth, Los Angeles",34.259571,-118.602325,1.0,Mexican Restaurant,Gym / Fitness Center,Creperie,Cajun / Creole Restaurant,Spa


In [190]:
df.dropna(inplace = True)
df.head()


,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Arleta, Los Angeles",34.241327,-118.432205,5.0,Movie Theater,Historic Site,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop
1,"Burbank, California",34.181648,-118.325855,1.0,Rental Service,Sandwich Place,Spa,Fast Food Restaurant,Concert Hall
3,"Canoga Park, Los Angeles",34.201078,-118.597826,1.0,Rental Car Location,Asian Restaurant,Thrift / Vintage Store,Pharmacy,Café
4,"Chatsworth, Los Angeles",34.259571,-118.602325,1.0,Mexican Restaurant,Gym / Fitness Center,Creperie,Cajun / Creole Restaurant,Spa
5,"Encino, Los Angeles",34.159132,-118.501617,1.0,Japanese Restaurant,Pizza Place,ATM,Gym,Chinese Restaurant


In [191]:
# show the map of clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
rainbow = ['red','blue','yellow','green','orange','cyan','magenta']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhoods'], df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### explore each cluster

In [207]:
#cluster 1 
df.loc[df['Cluster Labels'] == 0, df.columns[[0] + list(range(3, df.shape[1]))]]

,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,"Lake View Terrace, Los Angeles",0.0,Garden,Trail,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop


In [208]:
#cluster 2
df.loc[df['Cluster Labels'] == 1, df.columns[[0] + list(range(3, df.shape[1]))]]

,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"Burbank, California",1.0,Rental Service,Sandwich Place,Spa,Fast Food Restaurant,Concert Hall
3,"Canoga Park, Los Angeles",1.0,Rental Car Location,Asian Restaurant,Thrift / Vintage Store,Pharmacy,Café
4,"Chatsworth, Los Angeles",1.0,Mexican Restaurant,Gym / Fitness Center,Creperie,Cajun / Creole Restaurant,Spa
5,"Encino, Los Angeles",1.0,Japanese Restaurant,Pizza Place,ATM,Gym,Chinese Restaurant
6,"Glendale, California",1.0,Coffee Shop,Grocery Store,Discount Store,Automotive Shop,Shopping Plaza
7,"Granada Hills, Los Angeles",1.0,Pizza Place,Cosmetics Shop,Bakery,Insurance Office,Middle Eastern Restaurant
9,"Lake Balboa, Los Angeles",1.0,Park,Baseball Field,American Restaurant,Golf Course,Athletics & Sports
11,"Mission Hills, Los Angeles",1.0,Burger Joint,Thai Restaurant,Coffee Shop,Sandwich Place,Donut Shop
12,"North Hollywood, Los Angeles",1.0,Mexican Restaurant,American Restaurant,Gay Bar,Discount Store,Residential Building (Apartment / Condo)
13,"Northridge, Los Angeles",1.0,Mexican Restaurant,Hookah Bar,Hot Dog Joint,Middle Eastern Restaurant,Chinese Restaurant


In [206]:
#cluster 3 
df.loc[df['Cluster Labels'] == 2, df.columns[[0] + list(range(3, df.shape[1]))]]

,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
16,"Porter Ranch, Los Angeles",2.0,Gym,Yoga Studio,Gas Station,Furniture / Home Store,Frozen Yogurt Shop


In [209]:
#cluster 4 
df.loc[df['Cluster Labels'] == 3, df.columns[[0] + list(range(3, df.shape[1]))]]

,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,"West Hills, Los Angeles",3.0,Bakery,Park,Business Service,Donut Shop,English Restaurant


In [205]:
#cluster 5
df.loc[df['Cluster Labels'] == 4, df.columns[[0]+list(range(3, df.shape[1]))]]

,Neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,"Sylmar, Los Angeles",4.0,Pizza Place,Food Truck,Food,Mexican Restaurant,Yoga Studio


In [210]:
#cluster 6 
df.loc[df['Cluster Labels'] == 5, df.columns[[1] + list(range(3, df.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,34.241327,5.0,Movie Theater,Historic Site,Filipino Restaurant,Furniture / Home Store,Frozen Yogurt Shop


In [211]:
#cluster 7 
df.loc[df['Cluster Labels'] == 6, df.columns[[1] + list(range(3, df.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,34.164091,6.0,Pool,Home Service,Yoga Studio,Frozen Yogurt Shop,Fried Chicken Joint


#### Reference: 
Lab: Segmenting and Clustering Neighborhoods in New York City Provided by IBM Applied Data Science Capstone Course from Coursera